In [4]:
!pip install openai elasticsearch tqdm ipywidgets


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import json
import minsearch

with open("documents.json","r") as docs:
    documents_raw = json.load(docs)

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# index = minsearch.Index(
#     text_fields=["question", "text", "section"],
#     keyword_fields=["course"]
# )

# index.fit(documents)

In [2]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm 

In [3]:
es_client = Elasticsearch('http://localhost:9200')
es_client.ping()

True

In [5]:
def elastic_search_settings(index_name):
    index_settings = {
        "settings":{
            "number_of_shards":1,
            "number_of_replicas":0
        },
        "mappings":{
            "properties":{
                "text":{"type":"text"},
                "section":{"type":"text"},
                "question":{"type":"text"},
                "course":{"type":"keyword"},
            }
        }
    }

    return es_client.indices.create(index=index_name,body=index_settings)

In [7]:
index_name = "course-questions"

# elastic_search_settings(index_name)

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [9]:
def elastic_query(query):
    search_query = {
        "size":10,
        "query":{
            "bool":{
                "must":{
                    "multi_match":{
                        "query":query,
                        "fields":["question^3","text","section"],
                        "type":"best_fields"
                    }
                },
                "filter":{
                    "term":{
                        "course":"data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name,body=search_query)
    
    results = []

    for doc in response['hits']['hits']:
        results.append(doc['_source']) 

    return results

In [10]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    # required but ignored
    api_key='ollama',
)

In [11]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION. If fact not present, return NONE

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [12]:
def rag(query):
    search_results = elastic_query(query)
    prompt = build_prompt(query, search_results)
    print(prompt[0])
    answer = llm(prompt)
    return answer

In [13]:
rag("How do I start a docker container?")

Y


' To start a Docker container using the context provided in Module 1: Docker and Terraform FAQs, follow these steps to execute within their system?\n\nFirstly, run this command with appropriate settings for your operating environment as detailed below:\n\n```bash\nwinpty docker run -it --rm \\\n--name course-docker-container \\\n-p 80:80 \\\npython:3.9 \\\n/bin/true; exit \\$?\n```\nThis command does the following, explained in steps:\n1. `winpty`: This is a workaround for Windows and WSL environments where Docker commands sometimes fail to work correctly due to terminal limitations on line length or output buffering issues with bash-like shells like Git Bash or Cygwin’s sh.exe (not needed if you\'re working within an environment that handles these, such as the ones described in your answer).\n2. `docker run`: This is starting a new Docker container from image named "python:3.9". Replace with any appropriate base image for this course-specific instructions where necessary or adaptable 

In [14]:
print(_)

 To start a Docker container using the context provided in Module 1: Docker and Terraform FAQs, follow these steps to execute within their system?

Firstly, run this command with appropriate settings for your operating environment as detailed below:

```bash
winpty docker run -it --rm \
--name course-docker-container \
-p 80:80 \
python:3.9 \
/bin/true; exit \$?
```
This command does the following, explained in steps:
1. `winpty`: This is a workaround for Windows and WSL environments where Docker commands sometimes fail to work correctly due to terminal limitations on line length or output buffering issues with bash-like shells like Git Bash or Cygwin’s sh.exe (not needed if you're working within an environment that handles these, such as the ones described in your answer).
2. `docker run`: This is starting a new Docker container from image named "python:3.9". Replace with any appropriate base image for this course-specific instructions where necessary or adaptable to different require